In [18]:
%reload_ext autoreload
%autoreload 2

import socket
import psutil
import sys 
import os
from typing import Any

lib_path = ['/fsx_0/user/tranx/experiments']
import_paths = [
    "/fsx_0/user/tranx/rsync",
    # "/fsx_0/user/tranx/experiments/lib"
]
for path in import_paths:
    if path not in sys.path:
        sys.path.append(path)
from llm_mm_aligner.experiments.aws import launch_job
# import utils
    
hostname = socket.gethostname()
print("Host name:", hostname)
num_cpus = psutil.cpu_count()
print("Number of CPUs:", num_cpus)
total_memory = psutil.virtual_memory().total / (1024 ** 3)
print("Total memory (GB):", round(total_memory, 2))

import torch
from torch import nn 
from torch.nn import functional as F


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

Host name: submit-1
Number of CPUs: 32
Total memory (GB): 247.74


In [36]:
def run(
    config: str,
    overrides_dict: None,
    nodes: int = 1,
    config_base: str = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws",
    qos: str = "ar-ai-hipri",
    conda_env: str = "/fsx_0/user/ahmadyan/.conda/envs/aligner_20240822"
):
    config_file = os.path.join(config_base, config)
    
    overrides = [[
        ("slurm_args.qos", qos),
        ("slurm_args.account", qos),
        ("slurm_args.nodes", nodes)
    ]]
    
    if overrides_dict is not None:
        for k, v in overrides_dict.items():
            overrides[0].append((
                k, v
            ))
    
    launch_job.run_job(
        config_file = config_file,
        conda_env = conda_env,
        overrides = overrides
    )
    

# Stage 1: Test limit on batch_size

In [37]:
run(
    config="mm10.1_moe/moe/MH22final_70B_ViTH_336px_R1_moe_22x8x2_bz_8x1.json",
    nodes=8,
    overrides_dict={
        "trainer_args.output_dir": "/fsx_0/checkpoints/tranx/moe/70B_moe_22x8x2_n8_bz_8x1"
    }
)

In [40]:
base_config = "mm10.1_moe/moe/MH22final_70B_ViTH_336px_R1_moe_22x8x2_bz_8x1.json"

In [41]:
run(
    config=base_config,
    nodes=8,
    overrides_dict={
        "trainer_args.gradient_accumulation_steps": 2,
        "trainer_args.output_dir": "/fsx_0/checkpoints/tranx/moe/70B_moe_22x8x2_n8_bz_8x2"
    }
)

In [ ]:
for bz in [16, 32]:
    run(
        config=f"mm10.1_moe/moe/MH22final_70B_ViTH_336px_R1_moe_22x8x2_bz_{bz}x1.json",
        nodes=8,
        overrides_dict={
            "trainer_args.output_dir": "/fsx_0/checkpoints/tranx/moe/70B_moe_22x8x2_n8_bz_8x1"
        }
    )